In [ ]:
import cv2
import numpy as np
import random
import os

# --- Загружаем изображения ---
# Путь к папке с каплями
drop_folder = r'...'

# Загружаем все изображения капель из папки
drop_files = [os.path.join(drop_folder, f) for f in os.listdir(drop_folder) if f.endswith('.png')]
drops = [cv2.imdecode(np.fromfile(f, dtype=np.uint8), cv2.IMREAD_COLOR) for f in drop_files]

# Путь к папке с фоном
background_folder = r'...'

# Загружаем все изображения фонов из папки
background_files = [os.path.join(background_folder, f) for f in os.listdir(background_folder) if f.endswith('.jpg')]
backgrounds = [cv2.imdecode(np.fromfile(f, dtype=np.uint8), cv2.IMREAD_COLOR) for f in background_files]

# --- Фильтрация фона ---
blur_kernel_size = 9  # Размер ядра для размытия

# Параметры
num = 16 # Сохранение 
drop_id = 1  # Выбор капли (индекс в списке drops)
background_id = 3  # Выбор фона (индекс в списке backgrounds)
num_drops = 800  # Количество капель
min_scale = 0.01  # Минимальный масштаб капли
max_scale = 0.2  # Максимальный масштаб капли
max_attempts = 100  # Максимальное количество попыток для размещения капли

# Проверка индексов
if drop_id >= len(drops) or background_id >= len(backgrounds):
    raise ValueError("Некорректный индекс капли или фона. Убедитесь, что индексы находятся в допустимых пределах.")

# Выбор капли и фона
drop = drops[drop_id]
background = backgrounds[background_id]

# Размытие фона
background = cv2.GaussianBlur(background, (blur_kernel_size, blur_kernel_size), 0)

# Создаем копию фона для модификаций
result_image = background.copy()
combined_mask = np.zeros_like(background[:, :, 0], dtype=np.uint8)  # Маска для проверки пересечений

num_drops_un = 0
for _ in range(num_drops):
    for attempt in range(max_attempts):
        # Случайный масштаб и размер
        scale = random.uniform(min_scale, max_scale)
        new_w = int(drop.shape[1] * scale)
        new_h = int(drop.shape[0] * scale)
        
        # Случайные координаты (с учетом границ)
        x = random.randint(0, background.shape[1] - new_w)
        y = random.randint(0, background.shape[0] - new_h)
        
        # Проверяем, свободна ли область для новой капли
        if np.sum(combined_mask[y:y+new_h, x:x+new_w]) == 0:
            # Масштабируем каплю
            resized_drop = cv2.resize(drop, (new_w, new_h))
            
            # Создаем маску для капли (белый фон = 255, капля = 0)
            mask_drop = cv2.cvtColor(resized_drop, cv2.COLOR_BGR2GRAY)  # Преобразуем в градации серого
            mask_drop = np.where(mask_drop == 255, 0, 1).astype(np.uint8)  # Инвертируем маску: фон = 0, капля = 1
            
            # Область интереса на фоне
            roi = result_image[y:y+new_h, x:x+new_w]
            
            # Наложение капли
            result_roi = np.where(mask_drop[:, :, None], resized_drop, roi)
            result_image[y:y+new_h, x:x+new_w] = result_roi
            
            # Обновление общей маски
            combined_mask[y:y+new_h, x:x+new_w] = np.maximum(combined_mask[y:y+new_h, x:x+new_w], mask_drop)
            break
    else:
        print(f"Не удалось разместить каплю после {max_attempts} попыток.")
        num_drops_un += 1

drops = num_drops - num_drops_un
print(f"Всего капель на изображении: {drops}")

# Преобразование масок для визуализации
binary_mask_visual = combined_mask * 255

# Отображение результатов
cv2.namedWindow('Result Image', cv2.WINDOW_NORMAL)
cv2.namedWindow('Combined Mask', cv2.WINDOW_NORMAL)
cv2.imshow('Result Image', result_image)
cv2.imshow('Combined Mask', binary_mask_visual)
cv2.waitKey(0)
cv2.destroyAllWindows()



# Сохранение результатов 
cv2.imwrite(rf"... {num} .png", result_image)
cv2.imwrite(rf"... {num} .png", binary_mask_visual)